In [2]:
import os
from github import Github
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
from tqdm import tqdm
import time
import json
from multiprocessing.pool import ThreadPool
from github.GithubException import RateLimitExceededException
from urllib.request import urlopen
from io import BytesIO
from zipfile import ZipFile
import datetime
import json
import pinecone    
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter, Language
from langchain.docstore.document import Document
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Pinecone

### Load all repos depending on FastAPI

In [3]:

repo = "tiangolo/fastapi"
url = 'https://github.com/{}/network/dependents'.format(repo)
nextExists = True
result = []
while nextExists:
    try: 
        r = requests.get(url)
        soup = BeautifulSoup(r.content, "html.parser")

        result = result + [
            "{}/{}".format(
                t.find('a', {"data-repository-hovercards-enabled":""}).text,
                t.find('a', {"data-hovercard-type":"repository"}).text
            )
            for t in soup.findAll("div", {"class": "Box-row"})
        ]
        nextExists = False
        for u in soup.find("div", {"class":"paginate-container"}).findAll('a'):
            if u.text == "Next":
                nextExists = True
                url = u["href"]

        with open("data/.meta/dependents.json", "w") as f:
            json.dump(result, f, indent=2)
    except:
        print("Error")
        nextExists = True

    print(f"Has {len(result)} dependents")

    time.sleep(2)

with open("data/.meta/dependents.json", "w") as f:
    json.dump(result, f, indent=2)

Has 30 dependents
Has 60 dependents
Has 90 dependents
Has 120 dependents
Has 150 dependents
Has 180 dependents
Has 210 dependents
Has 240 dependents
Has 270 dependents
Has 300 dependents
Has 330 dependents
Has 360 dependents
Has 390 dependents
Has 420 dependents
Has 450 dependents
Has 480 dependents
Has 510 dependents
Has 540 dependents
Has 570 dependents
Has 600 dependents
Has 630 dependents
Has 660 dependents
Has 690 dependents
Has 720 dependents
Has 750 dependents
Has 780 dependents
Has 810 dependents
Has 840 dependents
Has 870 dependents
Has 900 dependents
Has 929 dependents
Has 959 dependents
Has 989 dependents
Has 1019 dependents
Has 1049 dependents
Has 1079 dependents
Has 1109 dependents
Has 1139 dependents
Has 1169 dependents
Has 1199 dependents
Has 1229 dependents
Has 1259 dependents
Has 1289 dependents
Has 1319 dependents
Has 1349 dependents
Has 1379 dependents
Has 1408 dependents
Has 1438 dependents
Has 1468 dependents
Has 1498 dependents
Has 1528 dependents
Has 1558 depende

In [ ]:
with open("data/.meta/dependents.json", "w") as f:
    json.dump(result, f, indent=2)

In [14]:
with open("data/_meta/dependents.json", "r") as f:
    repos_dependent_on_library = json.load(f)

In [17]:
print(len(repos_dependent_on_library))
print(repos_dependent_on_library[:2])

198744
['HAL9KKK/FC2', 'AndriyDykan/HWW14']


### Get top repos containing "fastapi"

In [3]:
github_api_key = os.environ.get('GITHUB_API_KEY')
assert github_api_key is not None, "You need to set your GITHUB_API_KEY environment variable."
g = Github(github_api_key, retry=None)

In [6]:
library_name = "tiangolo/fastapi"

# Search for repositories using the specified library
query = f'{library_name} in:readme,description'
result = g.search_repositories(query, sort="stars", order='desc')
result.totalCount

1000

In [7]:
found_repos = list(result)
found_repos[:2]

[Repository(full_name="tiangolo/fastapi"),
 Repository(full_name="tiangolo/full-stack-fastapi-postgresql")]

In [12]:
result

In [42]:
repos_df = pd.DataFrame([
    {
        "id": r.full_name,
        "stars": r.stargazers_count,
        "forks": r.forks_count,
        "watchers": r.watchers_count,
        "language": r.language,
    }
    for r in found_repos
])

print(f"Has total of {repos_df.shape[0]} repos")

repos_df = repos_df[repos_df.language == "Python"]
repos_df = repos_df[repos_df.id.apply(lambda x: x in repos_dependent_on_library)]
repos_df = repos_df.sort_values("stars", ascending=False)
repos_df = repos_df.reset_index(drop=True)
print(f"Has total of {repos_df.shape[0]} repos after filtering")

repos_df = repos_df.head(10)

Has total of 1020 repos
Has total of 516 repos after filtering


In [43]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(repos_df.sort_values(by=['stars'], ascending=False))

,id,stars,forks,watchers,language
0,tiangolo/full-stack-fastapi-postgresql,13490,2465,13490,Python
1,tiangolo/sqlmodel,11269,525,11269,Python
2,tiangolo/uvicorn-gunicorn-fastapi-docker,2391,311,2391,Python
3,ExpDev07/coronavirus-tracker-api,1602,328,1602,Python
4,amisadmin/fastapi-amis-admin,712,114,712,Python
5,zhanymkanov/fastapi_production_template,619,79,619,Python
6,developmentseed/titiler,607,128,607,Python
7,microsoft/cookiecutter-spacy-fastapi,465,65,465,Python
8,wxy2077/fastapi-mysql-generator,420,128,420,Python
9,tiangolo/full-stack-fastapi-couchbase,419,87,419,Python


### Download and unzip these repos

In [44]:
def download_and_unzip(url, extract_to):
    http_response = urlopen(url)
    zipfile = ZipFile(BytesIO(http_response.read()))
    zipfile.extractall(path=extract_to)

def download_repo(repo_id, extract_to):
    try: 
        # Automatically downloads main if master doesn't exist
        download_and_unzip(f"https://github.com/{repo_id}/archive/refs/heads/master.zip", extract_to=extract_to)
    except:
        print(f"Error downloading repo {repo_id}")

os.makedirs("data", exist_ok=True)
download_repo("tiangolo/fastapi", extract_to="data")
for i, row in tqdm(repos_df.iterrows()): 
    download_repo(row['id'], extract_to="data")

10it [00:16,  1.69s/it]


### Ingest the files

In [45]:
all_files = []

for root, dirs, files in os.walk("data", topdown=True):
   if root in ['data/_meta', 'data']:
      continue

   files = [os.path.join(root, f) for f in files if f.endswith('.py') or f.endswith('.md')]
   all_files += files

len(all_files), all_files[:5]

(2614,
 ['data/fastapi_production_template-main/README.md',
  'data/fastapi_production_template-main/tests/conftest.py',
  'data/fastapi_production_template-main/tests/__init__.py',
  'data/fastapi_production_template-main/tests/auth/__init__.py',
  'data/fastapi_production_template-main/tests/auth/test_routes.py'])

In [46]:
texts = []
for file in all_files: 
    with open(file) as f:
        try:  
            file_content = f.read()
            if file_content:
                texts.append(Document(page_content=file_content, metadata={"filename": file}))
        except Exception as e:
            print(f"Error reading file {file}: {e}")

In [47]:
text_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.PYTHON, chunk_size=800, chunk_overlap=100
)
chunks = text_splitter.split_documents(texts)

approx_tokens = sum([len(t.page_content) for t in chunks])
print(f"Embedding cost: {approx_tokens / 1000 * 0.0001}$")

Embedding cost: 1.0484773$


In [53]:
pinecone_api_key = os.environ.get("PINECONE_API_KEY", None)
assert pinecone_api_key is not None, "Needs PINECONE_API_KEY"
pinecone.init(      
	api_key=pinecone_api_key,      
	environment='gcp-starter'      
)      
index = pinecone.Index('codemate')

embeddings = OpenAIEmbeddings()

index_name = "codemate"
if index_name not in pinecone.list_indexes():
    pinecone.create_index(name=index_name, metric="cosine", dimension=1536)
    
#index.delete(delete_all=True)

docsearch = Pinecone.from_documents(chunks, embeddings, index_name=index_name)